In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from datetime import datetime

In [2]:
df_customers = pd.read_csv("https://raw.githubusercontent.com/Vignesh-ViggyPiggy/Data-Science-Assignment-eCommerce-Transactions-Dataset/refs/heads/main/Customers.csv")
df_customers

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15
...,...,...,...,...
195,C0196,Laura Watts,Europe,2022-06-07
196,C0197,Christina Harvey,Europe,2023-03-21
197,C0198,Rebecca Ray,Europe,2022-02-27
198,C0199,Andrea Jenkins,Europe,2022-12-03


In [3]:
df_products = pd.read_csv("https://raw.githubusercontent.com/Vignesh-ViggyPiggy/Data-Science-Assignment-eCommerce-Transactions-Dataset/refs/heads/main/Products.csv")
df_products

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31
...,...,...,...,...
95,P096,SoundWave Headphones,Electronics,307.47
96,P097,BookWorld Cookbook,Books,319.34
97,P098,SoundWave Laptop,Electronics,299.93
98,P099,SoundWave Mystery Book,Books,354.29


In [4]:
df_transactions = pd.read_csv("https://raw.githubusercontent.com/Vignesh-ViggyPiggy/Data-Science-Assignment-eCommerce-Transactions-Dataset/refs/heads/main/Transactions.csv")
df_transactions

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68
...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86


In [5]:
df_temp = pd.merge(df_transactions,df_customers,on = "CustomerID",how="inner" )
df = pd.merge(df_temp,df_products,on = "ProductID",how = "inner")
df = df.drop("Price_y",axis=1)
df = df.rename(columns={"Price_x":"Price"})
df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,CustomerName,Region,SignupDate,ProductName,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics
...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,SoundWave Smartwatch,Electronics
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,SoundWave Smartwatch,Electronics
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,SoundWave Smartwatch,Electronics
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,SoundWave Smartwatch,Electronics


In [6]:
totalValue = df["TotalValue"]
totalValueCategorical = pd.cut(x=totalValue, bins=range(0,int(totalValue.max())+100,100),labels=range(100,int(totalValue.max())+100,100))
df["totalValueCategorical"] = totalValueCategorical
df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,CustomerName,Region,SignupDate,ProductName,Category,totalValueCategorical
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,400
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,400
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,400
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,700
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,SoundWave Smartwatch,Electronics,500
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,SoundWave Smartwatch,Electronics,1400
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,SoundWave Smartwatch,Electronics,1900
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,SoundWave Smartwatch,Electronics,1000


In [7]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['SignupDate'] = pd.to_datetime(df['SignupDate'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   TransactionID          1000 non-null   object        
 1   CustomerID             1000 non-null   object        
 2   ProductID              1000 non-null   object        
 3   TransactionDate        1000 non-null   datetime64[ns]
 4   Quantity               1000 non-null   int64         
 5   TotalValue             1000 non-null   float64       
 6   Price                  1000 non-null   float64       
 7   CustomerName           1000 non-null   object        
 8   Region                 1000 non-null   object        
 9   SignupDate             1000 non-null   datetime64[ns]
 10  ProductName            1000 non-null   object        
 11  Category               1000 non-null   object        
 12  totalValueCategorical  1000 non-null   category      
dtypes: c

In [8]:
df['CustomerTenure'] = (df['TransactionDate'] - df['SignupDate']).dt.days
df['CustomerTenure'] = df['CustomerTenure'].apply(lambda x: x if x >= 0 else 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   TransactionID          1000 non-null   object        
 1   CustomerID             1000 non-null   object        
 2   ProductID              1000 non-null   object        
 3   TransactionDate        1000 non-null   datetime64[ns]
 4   Quantity               1000 non-null   int64         
 5   TotalValue             1000 non-null   float64       
 6   Price                  1000 non-null   float64       
 7   CustomerName           1000 non-null   object        
 8   Region                 1000 non-null   object        
 9   SignupDate             1000 non-null   datetime64[ns]
 10  ProductName            1000 non-null   object        
 11  Category               1000 non-null   object        
 12  totalValueCategorical  1000 non-null   category      
 13  Cust

In [9]:
features = df[["CustomerID", "Quantity", "TotalValue", "Price", "CustomerTenure", "Region", "Category"]]

encoder = OneHotEncoder(sparse_output=False)
encoded_cats = encoder.fit_transform(features[["Region", "Category"]])

encoded_cats_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(["Region", "Category"]))

features_final = pd.concat([features.drop(columns=["Region", "Category"]), encoded_cats_df], axis=1)

customer_features = features_final.drop_duplicates(subset=["CustomerID"]).set_index("CustomerID")



In [10]:
df["TransactionDate"] = pd.to_datetime(df["TransactionDate"])

transaction_frequency = df.groupby("CustomerID")["TransactionID"].nunique().reset_index()
transaction_frequency.rename(columns={"TransactionID": "TransactionFrequency"}, inplace=True)

latest_date = df["TransactionDate"].max()
recency = df.groupby("CustomerID")["TransactionDate"].max().reset_index()
recency["Recency"] = (latest_date - recency["TransactionDate"]).dt.days
recency.drop(columns=["TransactionDate"], inplace=True)

customer_features = customer_features.merge(transaction_frequency, on="CustomerID")
customer_features = customer_features.merge(recency, on="CustomerID")



In [11]:
customer_features

,CustomerID,Quantity,TotalValue,Price,CustomerTenure,Region_Asia,Region_Europe,Region_North America,Region_South America,Category_Books,Category_Clothing,Category_Electronics,Category_Home Decor,TransactionFrequency,Recency
0,C0199,1,300.68,300.68,631,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4,63
1,C0146,1,300.68,300.68,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,9
2,C0127,1,300.68,300.68,21,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,6,99
3,C0087,2,601.36,300.68,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7,105
4,C0070,3,902.04,300.68,737,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,C0058,1,396.34,396.34,4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,330
195,C0095,2,32.16,16.08,309,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2,89
196,C0151,4,223.96,55.99,32,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,3
197,C0078,2,995.52,497.76,26,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,18


In [12]:
customer_features = features_final.drop_duplicates(subset=["CustomerID"]).set_index("CustomerID")
scaler = StandardScaler()
customer_features_scaled = scaler.fit_transform(customer_features)

In [13]:
svd = TruncatedSVD(n_components=10, random_state=42)
svd_features = svd.fit_transform(customer_features_scaled)
svd_similarity = cosine_similarity(svd_features)
lookalike_dict = {}
customers = customer_features.index.tolist()
customers.sort()
for idx, customer in enumerate(customers[:20]):
  similar_indices = np.argsort(svd_similarity[idx])[::-1][1:4]
  lookalikes = [(customers[i], svd_similarity[idx][i]) for i in similar_indices]
  lookalike_dict[customer] = lookalikes
lookalike_df = pd.DataFrame({'CustomerID': customers[:20], 'Lookalikes': lookalike_dict.values()})
lookalike_df.to_csv('Lookalike.csv', index=False)

In [14]:
customer_features_scaled[customer_features_scaled<0]=0


In [15]:
nmf = NMF(n_components=10, random_state=42)
nmf_features = nmf.fit_transform(customer_features_scaled)
nmf_similarity = cosine_similarity(nmf_features)
lookalike_dict = {}
customers = customer_features.index.tolist()
customers.sort()

for idx, customer in enumerate(customers[:20]):
  similar_indices = np.argsort(nmf_similarity[idx])[::-1][1:4]
  lookalikes = [(customers[i], nmf_similarity[idx][i]) for i in similar_indices]
  lookalike_dict[customer] = lookalikes
lookalike_df = pd.DataFrame({'CustomerID': customers[:20], 'Lookalikes': lookalike_dict.values()})
lookalike_df.to_csv('Lookalike2.csv', index=False)